In [75]:
%pylab inline
from datascience import *
matplotlib.style.use('ggplot')
import gensim
import nltk
nltk.download('words')
nltk.download('stopwords')
#nltk.download('punkt')

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package words to
[nltk_data]     /Users/edwinroland/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/edwinroland/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


`%matplotlib` prevents importing * from pylab and numpy


True

In [ ]:
# description of columns

In [78]:
metadata_tb = Table.read_table('txtlab_Novel150_English.csv')
fiction_path = 'txtalb_Novel150_English/'
docterm_list = []

def fast_tokenizer(text):
    from string import punctuation
    low_case = text.lower()
    no_punct = "".join([char for char in low_case if char not in punctuation])
    tokens = no_punct.split()
    return tokens

for filename in metadata_tb['filename']:
    with open(fiction_path+filename, 'r') as file_in:
        novel = file_in.read()
    #tokens = nltk.word_tokenize(novel)
    tokens = fast_tokenizer(novel)
    docterm_list.append(tokens)

In [ ]:
from nltk.corpus import stopwords, words

proper_names = [word.lower() for word in words.words() if word.istitle()]

# Create Dictionary
dictionary = gensim.corpora.dictionary.Dictionary(docterm_list)
dictionary.filter_extremes(no_below=5)

stop_ids = [_id for _id,count in dictionary.doc2bow(stopwords.words('english')+proper_names)]
not_names = [word.lower() for word in words.words() if word.istitle()==False and word.lower() not in stopwords.words('english')]
real_ids = [_id for _id,count in dictionary.doc2bow(not_names)]

dictionary.filter_tokens(bad_ids = stop_ids, good_ids = real_ids)
corpus = [dictionary.doc2bow(doc) for doc in docterm_list]

In [ ]:
import time
start = time.time()
lda_model = gensim.models.LdaModel(corpus, num_topics=50, alpha='auto', eta='auto', id2word=dictionary, iterations=2500, passes = 4)
end = time.time()
print(end-start)

In [ ]:
# Quick look at topics inferred
lda_model.show_topics(20)

In [ ]:
# Deeper look at particular topic

lda_model.show_topic(8, topn=20)

In [ ]:
# Deeper look but labels with id2word mapping
lda_model.get_topic_terms(8, topn=20)

In [ ]:
## EX. The 'topn' argument returns only the given number of terms
##     for each topic. Rewrite that argument to return all values

## EX. Return a list of all term values for topic 1. By default,
##     these are ordered by the probability associated with each term.
##     Instead, order the list according to the words id2word label

In [ ]:
# Measure of model's "fit" to data -- typically want value between -7 and -4
lda_model.log_perplexity(corpus)

In [ ]:
# Most present topics in corpus

lda_model.top_topics(corpus)

In [ ]:
# Most prominent topics in a given document
lda_model.get_document_topics(corpus[0])

In [ ]:
# Distribution of all topics over a document
lda_model.get_document_topics(corpus[0], minimum_probability=0)

In [ ]:
list_of_topicdocs = [lda_model.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[y[1] for y in x] for x in list_of_topicdocs]
labeled_columns = [['Topic '+str(i),[x[i] for x in list_of_probabilities]] for i in range(50)]
metatopic_tb = metadata_tb.with_columns(labeled_columns)

In [ ]:
def correlator(tb, col_1, col_2):
    import numpy as np
    col_1_in_su = [(x-np.mean(tb[col_1]))/np.std(tb[col_1]) for x in tb[col_1]]
    col_2_in_su = [(x-np.mean(tb[col_2]))/np.std(tb[col_2]) for x in tb[col_2]]
    col_mult = [col_1_in_su[i]*col_2_in_su[i] for i in range(len(col_1_in_su))]
    r = np.mean(col_mult)
    return r**2